### 보험사 RAG 구현

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

loader = PyMuPDFLoader('./data/약관_31043(01)_나에게맞춘생활종합보험2404_20241001.pdf')
docs = loader.load()

print(type(docs))

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # 내가 몇개의 문자 단위로 나눌 건지 (토큰 아님)
    chunk_overlap = 200 # 겹친 글자 수 정해줌 (context 애매하게 짤리지 않게)
    ) 

split_docs = splitter.split_documents(docs)

embedding = OpenAIEmbeddings()
embedded_docs = embedding.embed_documents([content.page_content for content in split_docs])

vectorstore = FAISS.from_documents(
    documents=split_docs,
    embedding=embedding
)

# 검색기 생성
retriever = vectorstore.as_retriever()
retriever


<class 'list'>
<class 'list'>
1113


In [3]:
from pdfminer.high_level import extract_text
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

# 1. pdfminer로 텍스트 추출
pdf_file_path = './data/약관_31043(01)_나에게맞춘생활종합보험2404_20241001.pdf'
text = extract_text(pdf_file_path)

# 2. 텍스트를 페이지별로 나누는 방법
# 예시에서는 페이지 구분이 없으므로 간단히 한 덩어리로 텍스트를 추출
# 만약 페이지 구분이 필요하다면, PDF에서 페이지마다 텍스트를 추출해 저장할 수 있습니다.

# 3. 문서 분할 (청크화)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 청크 크기 설정
    chunk_overlap=100  # 약간의 오버랩을 두어 문맥 유지
)

# 텍스트를 청크화
split_docs = []
chunks = splitter.split_text(text)

# 청크화된 텍스트를 Document 객체로 변환
for i, chunk in enumerate(chunks):
    chunk_metadata = {
        "source": "약관",  # 소스는 '약관'으로 설정 (필요시 변경)
        "page": i + 1  # 페이지 정보 (이 경우 인덱스를 사용)
    }
    document = Document(page_content=chunk, metadata=chunk_metadata)
    split_docs.append(document)

# 4. 임베딩 생성
embedding = OpenAIEmbeddings()

# 5. 벡터 스토어 생성
faiss_db = FAISS.from_documents(split_docs, embedding)

# 6. 검색기 생성 (튜닝 포함)
retriever = faiss_db.as_retriever(search_kwargs={"k": 3})  # 관련 결과 3개만 반환

# 7. 증상 기반 검색 함수 정의
def search_insurance_terms(symptom):
    """
    주어진 증상(symptom)과 관련된 보험 약관을 검색.
    """
    results = retriever.get_relevant_documents(symptom)  # 증상 기반 검색
    for result in results:
        # 의미 있는 청크들만 출력
        if len(result.page_content) > 50 and "보험" in result.page_content:
            # 결과를 문장 단위로 출력 (적절한 길이로 자르기)
            summary = result.page_content[:500]  # 500자까지 출력
            summary = summary.replace('\n', ' ')  # 줄바꿈 제거
            print(f"관련 내용:\n{summary}...")
            print("-" * 50)

# 8. 테스트: 증상 검색
symptom_query = "아나팔락시스 보험"  # 간단하게 주요 키워드만 사용
search_insurance_terms(symptom_query)


관련 내용:
있습니다.  ①  아나필락시스쇼크진단비(최초1회한) ②  아나필락시스쇼크진단비(연간1회한)  󰊲 회사는  보험증권에  기재된  피보험자가  이  특별약관의  보험기간  중에  “아나필락시스 쇼크”로  진단  확정된  경우에는  아나필락시스쇼크진단비를  다음과  같이  지급합니다.  구분  아나필락시스 쇼크진단비  아나필락시스쇼크진단비 (최초1회한)  아나필락시스쇼크진단비 (연간1회한)  보험가입금액의  90%  보험가입금액의  10%  󰊳 위  󰊲의  “연간”이란,  보험기간  시작일부터  그날을  포함하여  매1년  단위로  도래하 는  계약해당일(최초  보험기간  시작일과  동일한  월,일을  말합니다)  전일까지의  기 간을  말합니다.  ⑥  간호ㆍ간병통합서비스  제공기관은  간호ㆍ간병통합서비스  제공인력의  근무  환경  및  처우  개선을  위하여  필요한  지원을  하여야  한다.  ⑦  국가  및  지방자치단체는  간호ㆍ간병통합서비스의  제공ㆍ확대,  간호ㆍ간병 통합서...
--------------------------------------------------
관련 내용:
130  아나필락시스쇼크진단비  보장예시  (가입금액  100만원  기준)  용어 풀이  2022.1.1 계약체결일자            2024.3.1     (첫  번째  진단일)      2026.1.1       (진단일)  2032.1.1 (갱신일자)  2032.5.1 (진단일)  세부보장  아나필락시스 쇼크진단비 (최초1회한)  90만원 지급  아나필락시스 쇼크진단비 (연간1회한)  10만원 지급  최초  1회  지급  후  소멸  10만원  지급  10만원 지급  연간  1회  한도로  계속  보장  아나필락시스 쇼크진단비  총100만원 지급  총10만원 지급  총10만원 지급  2.  (보험금  지급에  관한  세부규정) 󰊱 3.(“아나필락시스쇼크”의  정의  및  진단확정)의  “아나필락시스쇼크”의  진단은  임상적  특징  또는  혈액,  항

In [9]:
from pdfminer.high_level import extract_text
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

# 1. PDF에서 텍스트 추출
pdf_file_path = './data/약관_31043(01)_나에게맞춘생활종합보험2404_20241001.pdf'
text = extract_text(pdf_file_path)

# 2. 텍스트를 페이지별로 나누는 방법
# 페이지 구분이 필요하다면 페이지마다 텍스트를 추출하여 분리 가능
# (예시에서는 페이지 구분 없이 한 덩어리로 처리)

# 3. 텍스트를 청크화 (문서 분할)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 청크 크기 설정
    chunk_overlap=100  # 약간의 오버랩을 두어 문맥 유지
)

# 텍스트를 청크화하여 문서 목록 생성
split_docs = []
chunks = splitter.split_text(text)

# 청크화된 텍스트를 Document 객체로 변환
for i, chunk in enumerate(chunks):
    chunk_metadata = {
        "source": "약관",  # 소스 정보 설정
        "page": i + 1  # 페이지 정보 설정
    }
    document = Document(page_content=chunk, metadata=chunk_metadata)
    split_docs.append(document)

# 4. 임베딩 생성
embedding = OpenAIEmbeddings()

# 5. 벡터 스토어 생성
faiss_db = FAISS.from_documents(split_docs, embedding)

# 6. 검색기 생성 (튜닝 포함)
retriever = faiss_db.as_retriever(search_kwargs={"k": 3})  # 관련 결과 3개만 반환

# 7. 증상 기반 검색 함수 정의
def search_insurance_terms(symptom):
    """
    주어진 증상(symptom)과 관련된 보험 약관을 검색하고,
    가장 관련성 높은 문서 하나만 추출하는 함수.
    """
    results = retriever.get_relevant_documents(symptom)  # 증상 기반 검색
    if results:
        # 첫 번째 결과는 가장 유사도가 높은 문서
        best_result = results[0]
        
        # 결과 출력
        summary = best_result.page_content[:500]  # 500자까지 출력
        summary = summary.replace('\n', ' ')  # 줄바꿈 제거
        print(f"관련 내용:\n{summary}...")
    else:
        print("관련된 내용이 없습니다.")

# 8. 테스트: 증상 검색
symptom_query = "아나팔락시스 보험"  # 간단한 주요 키워드로 검색
search_insurance_terms(symptom_query)


관련 내용:
있습니다.  ①  아나필락시스쇼크진단비(최초1회한) ②  아나필락시스쇼크진단비(연간1회한)  󰊲 회사는  보험증권에  기재된  피보험자가  이  특별약관의  보험기간  중에  “아나필락시스 쇼크”로  진단  확정된  경우에는  아나필락시스쇼크진단비를  다음과  같이  지급합니다.  구분  아나필락시스 쇼크진단비  아나필락시스쇼크진단비 (최초1회한)  아나필락시스쇼크진단비 (연간1회한)  보험가입금액의  90%  보험가입금액의  10%  󰊳 위  󰊲의  “연간”이란,  보험기간  시작일부터  그날을  포함하여  매1년  단위로  도래하 는  계약해당일(최초  보험기간  시작일과  동일한  월,일을  말합니다)  전일까지의  기 간을  말합니다.  ⑥  간호ㆍ간병통합서비스  제공기관은  간호ㆍ간병통합서비스  제공인력의  근무  환경  및  처우  개선을  위하여  필요한  지원을  하여야  한다.  ⑦  국가  및  지방자치단체는  간호ㆍ간병통합서비스의  제공ㆍ확대,  간호ㆍ간병 통합서...


In [10]:
symptom_query = "독성 동물 접촉 보험"  # 간단하게 주요 키워드만 사용
search_insurance_terms(symptom_query)

관련 내용:
5.  (준용규정) 이  특별약관에  정하지  않은  사항은  보통약관을  따릅니다.  단,  이  특별약관에서는  보 통약관에서  정한  9.(만기환급금의  지급)의  만기환급금  및  37.(중도인출금)의  중도인출 금은  지급하지  않습니다.  27 독액성동물접촉중독진단비(연간1회한)  특별약관  1.  (보험금의  지급사유) 󰊱 회사는  보험증권에  기재된  피보험자가  「이  특별약관의  보험기간」  중에  「독액성  동 물과의  접촉으로  인한  중독」으로  진단  확정된  경우에는  연간  1회에  한하여  이  특 별약관의  보험가입금액을  독액성동물접촉중독진단비로  보험수익자에게  지급합니다. 󰊲 위  󰊱의  독액성동물접촉중독진단비는  “독액성  동물과의  접촉으로  인한  중독”의  직 접적인  원인,  중독의  종류  및  중독  부위와  상관없이  연간  1회를  초과하여  지급하 지  않습니다.  󰊳 위  󰊱의  “연간”이란,  보험기간  시작일부터  그날을 ...


###  챗봇 답변 적용

In [3]:
from pdfminer.high_level import extract_text
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI  # Import the correct model
from langchain.chains import LLMChain

# 1. PDF에서 텍스트 추출
pdf_file_path = './data/약관_31043(01)_나에게맞춘생활종합보험2404_20241001.pdf'
text = extract_text(pdf_file_path)

# 2. 텍스트를 페이지별로 나누는 방법
# 페이지 구분이 필요하다면 페이지마다 텍스트를 추출하여 분리 가능
# (예시에서는 페이지 구분 없이 한 덩어리로 처리)

# 3. 텍스트를 청크화 (문서 분할)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 청크 크기 설정
    chunk_overlap=100  # 약간의 오버랩을 두어 문맥 유지
)

# 텍스트를 청크화하여 문서 목록 생성
split_docs = []
chunks = splitter.split_text(text)

# 청크화된 텍스트를 Document 객체로 변환
for i, chunk in enumerate(chunks):
    chunk_metadata = {
        "source": "약관",  # 소스 정보 설정
        "page": i + 1  # 페이지 정보 설정
    }
    document = Document(page_content=chunk, metadata=chunk_metadata)
    split_docs.append(document)

# 4. 임베딩 생성
embedding = OpenAIEmbeddings()

# 5. 벡터 스토어 생성
faiss_db = FAISS.from_documents(split_docs, embedding)

# 6. 검색기 생성 (튜닝 포함)
retriever = faiss_db.as_retriever(search_kwargs={"k": 3})  # 관련 결과 3개만 반환

# 7. 증상 기반 검색 함수 정의 (자연스러운 답변 생성 추가)
def search_insurance_terms(symptom):
    """
    주어진 증상(symptom)과 관련된 보험 약관을 검색하고,
    가장 관련성 높은 문서를 추출한 뒤, 챗봇 스타일의 응답을 생성하는 함수.
    """
    results = retriever.get_relevant_documents(symptom)  # 증상 기반 검색
    if results:
        # 첫 번째 결과는 가장 유사도가 높은 문서
        best_result = results[0]
        
        # 결과 요약
        summary = best_result.page_content[:500]  # 500자까지 출력
        summary = summary.replace('\n', ' ')  # 줄바꿈 제거

        # 챗봇 응답을 위한 자연스러운 대화 생성
        # OpenAI GPT 모델을 사용하여 응답 생성
        prompt_template = """
        주어진 보험 약관에 대한 정보를 바탕으로 아래와 같이 질문에 대해 답변을 해주세요:
        
        사용자가 궁금해하는 주제: {symptom}
        관련 내용: {summary}
        
        질문에 대한 자연스럽고 대화식으로 답변을 생성해주세요.
        """
        
        prompt = PromptTemplate(input_variables=["symptom", "summary"], template=prompt_template)
        
        # Change OpenAI to ChatOpenAI
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)  # Use ChatOpenAI instead of OpenAI
        chain = LLMChain(llm=llm, prompt=prompt)
        
        # 자연스러운 대화식 답변 생성
        response = chain.run({"symptom": symptom, "summary": summary})

        print(f"답변: {response}")
    else:
        print("관련된 내용이 없습니다.")

# 8. 테스트: 증상 검색
symptom_query = "아나팔락시스 보험"  # 간단한 주요 키워드로 검색
search_insurance_terms(symptom_query)




C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17788\1788019336.py:75: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)  # Use ChatOpenAI instead of OpenAI


답변: 아나필락시스 보험에 대한 내용을 알려드릴게요. 아나필락시스 보험은 아나필락시스 쇼크 진단비를 지급하는 것으로 보입니다. 이 보험은 아나필락시스 쇼크로 진단 확정된 경우에 해당하며, 보험가입금액의 일부를 지급해줍니다. 연간 1회까지 지원되는데, 연간이란 보험기간 시작일부터 매년 계약해당일 전일까지를 의미합니다. 또한, 간호ㆍ간병통합서비스 제공기관은 제공인력의 근무 환경 및 처우 개선을 위해 지원을 하여야 한다고 합니다. 국가 및 지방자치단체도 간호ㆍ간병통합서비스의 제공 및 확대에 관심을 가지고 있습니다. 이렇게요. 더 궁금한 점 있으시면 언제든지 물어봐주세요.


In [4]:
symptom_query = "골절 관련된 보험에 대해 알려줘"  # 간단한 주요 키워드로 검색
search_insurance_terms(symptom_query)


답변: 골절 관련된 보험에 대해 알려드릴게요. 약관 요약서에 따르면, 보험가입자나 피보험자는 보험계약을 체결할 때 보험사에 고지사항을 부실하게 알려서는 안 된다는 의무가 있습니다. 이를 위반할 경우 보험사는 일정 요건 아래 계약을 해지할 수 있지만, 보험사가 해당 사실을 알고 있거나 중대한 과실로 인해 알지 못했을 경우에는 계약을 해지할 수 없습니다. 또한, 일정 기간이 경과한 경우에도 계약을 해지할 수 없다고 합니다. 따라서 골절과 관련된 보험을 가입하실 때에는 약관에 명시된 의무를 준수해야 합니다. 추가적인 질문이 있으시면 언제든지 물어봐주세요.


In [5]:
symptom_query = "교통사고에 대한 보험을 다양하게 알려줄수있을까"  # 간단한 주요 키워드로 검색
search_insurance_terms(symptom_query)

답변: 네, 교통사고에 대한 다양한 보험 상품이 있습니다. 교통사고처리지원금Ⅲ부터 교통사고처리지원금통합형까지 다양한 옵션이 제공되고 있어요. 이 중에서 영업용운전자 중상해제외로 되어 있는 상품들이 있습니다. 이러한 상품들은 교통사고로 인한 다양한 비용을 보상받을 수 있도록 도와주는데요. 교통사고로 인한 상황에 대비하여 적절한 보험을 가입해두는 것이 중요하니, 자세한 내용은 약관을 참고하시거나 보험상담사와 상담하여 보다 적합한 상품을 선택하시는 것이 좋을 것 같아요.


In [3]:
from pdfminer.high_level import extract_text
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

# 1. PDF에서 텍스트 추출
pdf_file_path = './data/약관_31043(01)_나에게맞춘생활종합보험2404_20241001.pdf'
text = extract_text(pdf_file_path)

# 2. 텍스트를 청크화 (문서 분할)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # 청크 크기 설정
    chunk_overlap=100  # 약간의 오버랩을 두어 문맥 유지
)

# 텍스트를 청크화하여 문서 목록 생성
split_docs = []
chunks = splitter.split_text(text)

# 청크화된 텍스트를 Document 객체로 변환
for i, chunk in enumerate(chunks):
    chunk_metadata = {
        "source": "약관",  # 소스 정보 설정
        "page": i + 1  # 페이지 정보 설정
    }
    document = Document(page_content=chunk, metadata=chunk_metadata)
    split_docs.append(document)

# 3. 임베딩 생성
embedding = OpenAIEmbeddings()

# 4. FAISS 벡터 스토어 생성
faiss_db = FAISS.from_documents(split_docs, embedding)

# 5. FAISS 인덱스 저장
index_folder_path = './data/new_faiss_index'
faiss_db.save_local(index_folder_path)
print(f"FAISS 인덱스가 '{index_folder_path}'에 저장되었습니다.")

# 6. FAISS 인덱스 로드
loaded_faiss_db = FAISS.load_local(index_folder_path, embedding)

# 7. 검색기 생성
retriever = loaded_faiss_db.as_retriever(search_kwargs={"k": 3})  # 관련 결과 3개만 반환

# 8. 증상 기반 검색 함수 정의
def search_insurance_terms(symptom):
    """
    주어진 증상(symptom)과 관련된 보험 약관을 검색하고,
    가장 관련성 높은 문서를 추출한 뒤, 챗봇 스타일의 응답을 생성하는 함수.
    """
    results = retriever.get_relevant_documents(symptom)  # 증상 기반 검색
    if results:
        # 첫 번째 결과는 가장 유사도가 높은 문서
        best_result = results[0]
        
        # 결과 요약
        summary = best_result.page_content[:500]  # 500자까지 출력
        summary = summary.replace('\n', ' ')  # 줄바꿈 제거

        # 챗봇 응답을 위한 자연스러운 대화 생성
        prompt_template = """
        주어진 보험 약관에 대한 정보를 바탕으로 아래와 같이 질문에 대해 답변을 해주세요:
        
        사용자가 궁금해하는 주제: {symptom}
        관련 내용: {summary}
        
        질문에 대한 자연스럽고 대화식으로 답변을 생성해주세요.
        """
        
        prompt = PromptTemplate(input_variables=["symptom", "summary"], template=prompt_template)
        
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)  # ChatOpenAI 사용
        chain = LLMChain(llm=llm, prompt=prompt)
        
        # 자연스러운 대화식 답변 생성
        response = chain.run({"symptom": symptom, "summary": summary})

        print(f"답변: {response}")
    else:
        print("관련된 내용이 없습니다.")

# 9. 테스트: 증상 검색
symptom_query = "아나팔락시스 보험"  # 간단한 주요 키워드로 검색
search_insurance_terms(symptom_query)



FAISS 인덱스가 './data/new_faiss_index'에 저장되었습니다.


ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).